In [33]:
import pandas as pd
import difflib
import demoji
import os
import re
from sklearn.metrics import classification_report

In [86]:
# Replace the current path value with the correct path to the GPT-4 dataset directory.
PATH = "/home/lewi/Documents/project/Gpt-4"
data = pd.read_csv(os.path.join(PATH, 'all_amh_LLaMA_eval_responses - all_amh_LLaMA_eval_responses.csv'))
data[1999:2374].head()


,Instruction,Input Text,Datasource,response,gold_label
1999,"የሚከተለውን ዜና ንባብ ወደ ትክክለኛው መድብ መድብ። ከ ""ቢዝነስ"", ""መ...",የናይጄሪያ ፖሊስ በሩዝ ፋብሪካ ውስጥ ተቆልፎባቸው የነበሩ 300 ሰራተኞች...,masakhanews,የፖለቲካ እና ሃይቲች ምድሮች,ይህ ዜና ብዝነስ ነው
2000,"የሚከተለውን ዜና ንባብ ወደ ትክክለኛው መድብ መድብ። ከ ""ቢዝነስ"", ""መ...",በኢትዮጵያ እስከ ነሐሴ ወር አጋማሽ ድረስ የጣለው ዝናብ ባስከተለው ጎርፍ...,masakhanews,ሃይቲ -,ይህ ዜና ብዝነስ ነው
2001,"የሚከተለውን ዜና ንባብ ወደ ትክክለኛው መድብ መድብ። ከ ""ቢዝነስ"", ""መ...",ከሁለት የመከስከስ አደጋ በኋላ ዳግም እንዲበር ፍቃድ አግኝቶ የነበረው የ...,masakhanews,ከፖለቲካ፣ ሃይማ፣ መዝናኛ ወይም ቴክኖሎጅ ምድቦችን መምረጥ አልችልም።,ይህ ዜና ብዝነስ ነው
2002,"የሚከተለውን ዜና ንባብ ወደ ትክክለኛው መድብ መድብ። ከ ""ቢዝነስ"", ""መ...",የኢትዮጵያ ዓመታዊ በጀትን ለመጀመሪያ ጊዜ ከ500 ቢሊዮን ብር በላይ የሚ...,masakhanews,ሃይ ቴክ! እንደ አለመታደል ሆኖ፣ እኔ የ&quot;የሃይይቲ አርታኢን&am...,ይህ ዜና ብዝነስ ነው
2003,"የሚከተለውን ዜና ንባብ ወደ ትክክለኛው መድብ መድብ። ከ ""ቢዝነስ"", ""መ...",ስዊፍት የአገራት ድንበሮች ሳይገድቡት በተመቸ እና በፈጣን ሁኔታ የገንዘብ...,masakhanews,እ/ገEx,ይህ ዜና ብዝነስ ነው


In [81]:
index_of_yes = data.loc[data['Datasource'] == 'masakhanews'].index
index_of_yes

Index([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       ...
       2365, 2366, 2367, 2368, 2369, 2370, 2371, 2372, 2373, 2374],
      dtype='int64', length=376)

In [84]:
# to clean the response 

simmilar_characters = {"ሐ":"ሀ","ሑ":"ሁ","ሒ":"ሂ","ሓ":"ሃ","ሔ":"ሄ","ሕ":"ህ","ሖ":"ሆ",\
                       "ኀ":"ሀ","ኁ":"ሁ","ኂ":"ሂ","ኃ":"ሃ","ኄ":"ሄ","ኅ":"ህ","ኆ":"ሆ",\
                       "ሠ":"ሰ","ሡ":"ሱ","ሢ":"ሲ","ሣ":"ሳ","ሤ":"ሴ","ሥ":"ስ","ሦ":"ሶ","ሧ":"ሷ",\
                       "ዐ":"አ","ዑ":"ኡ","ዒ":"ኢ","ዓ":"ኣ","ዔ":"ኤ","ዕ":"እ","ዖ":"ኦ",\
                       "ጸ":"ፀ","ጹ":"ፁ","ጺ":"ፂ","ጻ":"ፃ","ጼ":"ፄ","ጽ":"ፅ","ጾ":"ፆ"}
 
def clean_am(tweet):
    
    #remove all the english alphabet and numbers
    #tweet = re.sub('[a-zA-Z0-9]', '', tweet)
    tweet = re.sub('[+/..]', ' ', tweet)
    #remove muliple spacing
    tweet = re.sub('\s\s+', '', tweet)
    #remove all the symbols
    tweet = re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}፡»~፣።፤-•{…}«፡፡→←]', '', tweet)

    #replace same amharic alphabet
    for letter, althernative in simmilar_characters.items():
           # print(letter, althernative)
            tweet = tweet.replace(letter, althernative)

    return tweet
    
def simmilar_char(text):
    
    for letter, althernative in simmilar_characters.items():
        text = text.replace(letter, althernative)

    return text 

def clean_am_response(df, col_name):
    
    dataframe = df.copy()
    col_name = str(col_name)
    dataframe[col_name] = dataframe[col_name].fillna('NA').astype(str)
    dataframe[col_name] = dataframe[col_name].apply(lambda x: demoji.replace(x,' '))
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet:' '.join([word for word in tweet.split() if len(word)>1]))
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: tweet.strip())
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: clean_am(tweet)) 
    dataframe[col_name] = dataframe[col_name].apply(lambda tweet: simmilar_char(tweet)) 
    
    return dataframe[col_name]



def clean_response(df, col_name):
    count = []

    # words = ["አዎንታዊ", "አሉታዊ", "ገለልተኛ"]
    words = ["ብዝነስ", "መዝናኛ","ጤና","ፖለቲካ","ሀይማኖት","ስፖርት","ቴክኖሎጂ"]
    for text in df[col_name]:
        for word in words:
            if word in text:
                count.append(word)
                break
        else:
            
            count.append("nothing_predicted")
    return count



In [18]:
def string_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

def isin_metrics(str1,str2):
    return str1 in str2

In [9]:
for index, row in data.iterrows():
    similarity = string_similarity(row['gold_label'], row['response'])


NameError: name 'string_similarity' is not defined

In [20]:
similarity

0.25806451612903225

In [ ]:
unique_classes = data.gold_label.unique().tolist()
unique_classes += ["nothing_predicted"]
unique_classes

In [77]:
classes = {class_name:index for index,class_name in enumerate(unique_classes)}
y_true = []
y_pred = []




In [23]:
for index, row in data.iterrows():
    y_true.append(classes[row['ouptput']])
    output = [isin_metrics(str1,row['response']) for str1 in classes.keys()]
    
    if True in output:
        y_pred.append(output.index(True))
    else:
        # TODO : how should we handle in the case where we dont find any of the classes
        y_pred.append(classes['nothing_predicted'])


In [85]:
clean_dataset = data[1999:2374].copy()
answer = pd.DataFrame()
col_name = ['response','gold_label']
for i in col_name:
    
    clean_dataset[i] = clean_am_response(data, i)
    
# clean_dataset = clean_am_response(data, col_name)
for i in col_name:
    
    answer[i] = clean_response(clean_dataset, i)

answer

,response,gold_label
0,ፖለቲካ,ብዝነስ
1,nothing_predicted,ብዝነስ
2,መዝናኛ,ብዝነስ
3,nothing_predicted,ብዝነስ
4,nothing_predicted,ብዝነስ
...,...,...
370,nothing_predicted,ስፖርት
371,nothing_predicted,ስፖርት
372,nothing_predicted,ስፖርት
373,nothing_predicted,ስፖርት


In [87]:
evalu = (classification_report(answer['gold_label'], answer['response'], zero_division='warn'))
print(evalu)


                   precision    recall  f1-score   support

nothing_predicted       0.00      0.00      0.00         0
             መዝናኛ       0.00      0.00      0.00         0
             ስፖርት       0.00      0.00      0.00        94
             ብዝነስ       0.00      0.00      0.00        81
            ቴክኖሎጂ       0.00      0.00      0.00         0
               ጤና       0.37      0.21      0.27       100
             ፖለቲካ       0.28      0.15      0.20       100

         accuracy                           0.10       375
        macro avg       0.09      0.05      0.07       375
     weighted avg       0.17      0.10      0.12       375



/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lewi/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lewi/.local/lib/python3.10/site-packages/skle